<a href="https://colab.research.google.com/github/DalalAL-Alimi/IDA/blob/main/IDA_%E2%80%8ASVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns # I added this
sns.axes_style('whitegrid');
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, cohen_kappa_score
import seaborn as sn
import scipy.io as sio
import os


import pandas as pd
import numpy as np
import time
import seaborn as sn


## Read the Data

In [ ]:
def loadData(name):
    data_path = os.path.join(os.getcwd(),'../data')
    if name == 'IP':
        data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(os.path.join(data_path, 'Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(os.path.join(data_path, 'PaviaU.mat'))['paviaU']
        labels = sio.loadmat(os.path.join(data_path, 'PaviaU_gt.mat'))['paviaU_gt']
    elif name == 'KSC':
        data = sio.loadmat(os.path.join(data_path, 'KSC.mat'))['KSC']
        labels = sio.loadmat(os.path.join(data_path, 'KSC_gt.mat'))['KSC_gt'] 
    elif name == 'Di':
        data = sio.loadmat(os.path.join(data_path, 'Dioni.mat'))['Dioni']
        labels = sio.loadmat(os.path.join(data_path, 'Dioni_GT.mat'))['Dioni_gt']
    elif name == 'Lk':
        data = sio.loadmat(os.path.join(data_path, 'Loukia.mat'))['Loukia']
        labels = sio.loadmat(os.path.join(data_path, 'Loukia_GT.mat'))['Loukia_gt']
    return data, labels

In [ ]:
def second_step(X, orgin, numComponents):
  # standardize the features

  sc = StandardScaler()
  #X_std = sc.fit_transform(X)
  #orgin_std = sc.fit_transform(orgin)

  # Calculate the covariance matrix first
  
  correlation_matrix = np.corrcoef(orgin.T)
  #covariance_matrix1 = np.cov(orgin.T)
  covariance_matrix = np.cov(X.T)   # The result is a Positive semidefinite matrix
  print(correlation_matrix.shape, covariance_matrix.shape)
  final_matrix = covariance_matrix - correlation_matrix
  print(max(pd.DataFrame(final_matrix)))
  # Eigendecomposition of the covariance
  eigen_values, eigen_vectors = np.linalg.eig(final_matrix)
  # Calculate the variance explained on each of those components
  variance_explained = [(i/sum(eigen_values))*100 for i in eigen_values]
  # Calculate the cumulative sum of variance explained
  cumulative_variance_explained = np.cumsum(variance_explained)
  #Taking two components and create a projection matrix
  projection_matrix = (eigen_vectors.T[:][:numComponents]).T
  
  X_IDA1 = X.dot(projection_matrix)

  return X_IDA1

In [ ]:
def extract_pixels(X, y):
  q = X.reshape(-1, X.shape[2])
  df = pd.DataFrame(data = q)
  df = pd.concat([df, pd.DataFrame(data = y.ravel())], axis=1)
  df.columns= [f'band{i}' for i in range(1, 1+X.shape[2])]+['class']

  return df

In [ ]:
from skimage.morphology import reconstruction
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler

def IDA(X1, y1, band_no, numComponents):
    
  df = extract_pixels(X1, y1)
  X = df.iloc[:, :-1]
  y = df.iloc[:, -1]

  X_mean = X
  X_mean['mean'] = X_mean.mean(axis=1)
  Avr = [X_mean['mean']]
  rep_Avr = np.repeat(Avr, band_no)
  rep_Avr =np.reshape(rep_Avr, (-1,band_no))
  main_X = X.iloc[:, :-1]
  seed = main_X - rep_Avr
  dilated = reconstruction(seed.values, main_X.values)
  hdome = main_X - dilated
  X = second_step(hdome,X.iloc[:, :-1], numComponents)
  scaler = QuantileTransformer(n_quantiles=15, random_state=0,output_distribution='normal') #
  scaler = scaler.fit_transform(X)
 
  
  return scaler, y 

In [ ]:
name = 'IP'
X, y = loadData(name)
h1 = X.shape[0]
w1 = X.shape[1]
X.shape, y.shape

((145, 145, 200), (145, 145))

In [ ]:
import time
numComponents = 15
band_no = X.shape[2]
K = numComponents  # The target number of bands
training_start_time = time.time()
X1, y1 = IDA(X,y, band_no, numComponents)
training_end_time = time.time()
X1.shape, y1.shape

(200, 200) (200, 200)
199


((21025, 15), (21025,))

In [ ]:
print("preprocessing time : ", training_end_time - training_start_time )
preprocessing_time = training_end_time - training_start_time 

preprocessing time :  3.002713680267334


In [ ]:
q = pd.concat([pd.DataFrame(data = X1), pd.DataFrame(data = y1.ravel())], axis = 1)
q.columns = [f'PC-{i}' for i in range(1,numComponents+1)]+['class']

In [ ]:
q.head()

,PC-1,PC-2,PC-3,PC-4,PC-5,PC-6,PC-7,PC-8,PC-9,PC-10,PC-11,PC-12,PC-13,PC-14,PC-15,class
0,0.025774,0.815405,1.000387,-1.474532,-0.055058,-0.656865,-1.577724,-0.069136,0.314522,-1.472449,-0.832973,0.787559,-0.645685,-0.758229,-0.063932,3
1,0.540073,1.507526,1.504245,-1.078301,-0.523161,-1.491999,-0.827563,-0.844380,-1.148205,-0.905174,-0.782528,0.224212,-1.840352,1.269320,-1.398133,3
2,0.670626,1.550588,1.525545,-0.845244,-1.296709,-1.490121,0.159907,-1.467267,-1.487307,0.000114,-0.901984,0.291273,-1.916838,0.932605,1.502799,3
3,0.591166,1.523220,1.508136,-1.057504,-0.515309,-1.498901,-0.744577,-0.574355,-1.169382,-0.822484,-0.805126,0.248120,-1.831857,1.465688,-1.384654,3
4,0.245429,1.149828,1.111285,-1.499972,0.143163,-1.466619,-1.603703,0.182224,-0.038219,-1.504269,-1.227445,0.760402,-1.190484,0.841546,-0.076608,3


# split the dataset

In [ ]:
x = q[q['class'] != 0]

X = x.iloc[:, :-1].values

y = x.loc[:, 'class'].values 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80, random_state=345, stratify=y)

In [ ]:
X_train.shape

(2049, 15)

In [ ]:
X_test.shape

(8200, 15)

In [ ]:
svm =  SVC(C = 100, kernel = 'rbf', cache_size = 10*1024)
import time
start = time.time()
svm.fit(X_train, y_train)
end = time.time()

Train_time = end - start
print(Train_time)

0.16093206405639648


In [ ]:
test_start = time.time()
y_pred = svm.predict(X_test)
test_end = time.time()

TEST_TIME = test_end - test_start
print(TEST_TIME)

0.766690731048584


In [ ]:
if name == 'IP':
    names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
                    ,'Grass-pasture', 'Grass-trees', 'Grass-pasture-mowed', 
                    'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                    'Soybean-clean', 'Wheat', 'Woods', 'Buildings-Grass-Trees-Drives',
                    'Stone-Steel-Towers']
elif name == 'SA':
    names = ['Brocoli_green_weeds_1','Brocoli_green_weeds_2','Fallow','Fallow_rough_plow','Fallow_smooth',
                    'Stubble','Celery','Grapes_untrained','Soil_vinyard_develop','Corn_senesced_green_weeds',
                    'Lettuce_romaine_4wk','Lettuce_romaine_5wk','Lettuce_romaine_6wk','Lettuce_romaine_7wk',
                    'Vinyard_untrained','Vinyard_vertical_trellis']
elif name == 'PU':
    names = ['Asphalt','Meadows','Gravel','Trees', 'Painted metal sheets','Bare Soil','Bitumen',
                    'Self-Blocking Bricks','Shadows']
elif name == 'KSC':
        names = ["Scrub", "Willow swamp",
                        "Cabbage palm hammock", "Cabbage palm/oak hammock",
                        "Slash pine", "Oak/broadleaf hammock",
                        "Hardwood swamp", "Graminoid marsh", "Spartina marsh",
                        "Cattail marsh", "Salt marsh", "Mud flats", "Wate"]
    
elif name == 'Di':
    names = ["Scrub", "Willow swamp",
                    "Cabbage palm hammock", "Cabbage palm/oak hammock",
                    "Slash pine", "Oak/broadleaf hammock",
                    "Hardwood swamp", "Graminoid marsh", "Spartina marsh",
                    "Cattail marsh", "Salt marsh", "Mud flats"]
elif name == 'Lk':
    names = ["Dense Urban Fabric", "Mineral Extraction Sites",
                    "Non Irrigated Arable Land", "Fruit Trees",
                    "Olive Groves","Broad-Leaved Forest", "Coniferous Forest","Mixed Forest",
                    "Dense Sclerophyllous Vegetation", "Sparce Sclerophyllous Vegetation", "Sparcely Vegetated Areas",
                    "Rocks and Sand", "Water", "Coastal Water"]
    

In [ ]:
data = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(data, columns=np.unique(names), index = np.unique(names))
df_cm.index.name = 'Actual'
df_cm.columns.name = 'Predicted'
plt.figure(figsize = (10,8))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, cmap="Reds", annot=True,annot_kws={"size": 16}, fmt='d')
#plt.savefig('cmap.png', dpi=300)

<AxesSubplot: xlabel='Predicted', ylabel='Actual'>

In [ ]:
print(classification_report(y_test, y_pred, target_names = names))#60

                              precision    recall  f1-score   support

                     Alfalfa       0.82      0.76      0.79        37
                 Corn-notill       0.68      0.76      0.72      1143
                Corn-mintill       0.70      0.74      0.72       664
                        Corn       0.57      0.44      0.50       190
               Grass-pasture       0.85      0.89      0.87       386
                 Grass-trees       0.95      0.93      0.94       584
         Grass-pasture-mowed       0.48      0.55      0.51        22
               Hay-windrowed       0.97      0.96      0.96       382
                        Oats       0.47      0.50      0.48        16
              Soybean-notill       0.68      0.66      0.67       778
             Soybean-mintill       0.79      0.81      0.80      1964
               Soybean-clean       0.77      0.65      0.71       475
                       Wheat       0.96      0.99      0.98       164
                   

In [ ]:
clsf_report = pd.DataFrame(classification_report(y_test, y_pred, target_names = names, output_dict=True)).transpose()
clsf_report.to_csv('SVM_'+name+' Classification Report.csv', index= True)

In [ ]:
from operator import truediv
def AA_andEachClassAccuracy2(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [ ]:
import pandas as pd
import numpy as np
import time
import seaborn as sn
def reports2 (X_test,y_test,name):
    start = time.time()
    Y_pred = X_test#svm.predict(X_test)
    #y_pred = np.argmax(Y_pred, axis=1)
    end = time.time()

    TESTING_TIME = end - start
    print(TESTING_TIME)
    if name == 'IP':
        target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
                        ,'Grass-pasture', 'Grass-trees', 'Grass-pasture-mowed', 
                        'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                        'Soybean-clean', 'Wheat', 'Woods', 'Buildings-Grass-Trees-Drives',
                        'Stone-Steel-Towers']
    elif name == 'SA':
        target_names = ['Brocoli_green_weeds_1','Brocoli_green_weeds_2','Fallow','Fallow_rough_plow','Fallow_smooth',
                        'Stubble','Celery','Grapes_untrained','Soil_vinyard_develop','Corn_senesced_green_weeds',
                        'Lettuce_romaine_4wk','Lettuce_romaine_5wk','Lettuce_romaine_6wk','Lettuce_romaine_7wk',
                        'Vinyard_untrained','Vinyard_vertical_trellis']
    elif name == 'PU':
        target_names = ['Asphalt','Meadows','Gravel','Trees', 'Painted metal sheets','Bare Soil','Bitumen',
                        'Self-Blocking Bricks','Shadows']
    elif name == 'KSC':
            target_names = ["Scrub", "Willow swamp",
                            "Cabbage palm hammock", "Cabbage palm/oak hammock",
                            "Slash pine", "Oak/broadleaf hammock",
                            "Hardwood swamp", "Graminoid marsh", "Spartina marsh",
                            "Cattail marsh", "Salt marsh", "Mud flats", "Wate"]
    elif name == 'Di':
        target_names = ["Scrub", "Willow swamp",
                        "Cabbage palm hammock", "Cabbage palm/oak hammock",
                        "Slash pine", "Oak/broadleaf hammock",
                        "Hardwood swamp", "Graminoid marsh", "Spartina marsh",
                        "Cattail marsh", "Salt marsh", "Mud flats"]
    elif name == 'Lk':
        target_names = ["Dense Urban Fabric", "Mineral Extraction Sites",
                        "Non Irrigated Arable Land", "Fruit Trees",
                        "Olive Groves","Broad-Leaved Forest", "Coniferous Forest","Mixed Forest",
                        "Dense Sclerophyllous Vegetation", "Sparce Sclerophyllous Vegetation", "Sparcely Vegetated Areas",
                        "Rocks and Sand", "Water", "Coastal Water"]
    
    classification = classification_report(y_test, y_pred, target_names=target_names)
    oa = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    ###################################
    '''clsf_report = pd.DataFrame(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names, output_dict=True)).transpose()
    clsf_report.to_csv('Model2 Classification Report.csv', index= True)'''
    #########################
    df_cm = pd.DataFrame(confusion, columns=np.unique(target_names), index = np.unique(target_names))
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (10,8))
    sn.set(font_scale=1.4)#for label size
    sn.heatmap(df_cm, cmap="Reds", annot=True,annot_kws={"size": 16}, fmt='d')
    #plt.savefig('/results/confusion_matrix_model2.png', dpi=300)
    ##############
    each_acc, aa = AA_andEachClassAccuracy2(confusion)
    kappa = cohen_kappa_score(y_test, y_pred)
    '''score = svm.evaluate(X_test, y_test, batch_size=32)
    Test_Loss =  score[0]*100
    Test_accuracy = score[1]*100'''
    
    return TESTING_TIME, classification, confusion, oa*100, each_acc*100, aa*100, kappa*100

In [ ]:
TESTING_TIME, classification, confusion, oa, each_acc, aa, kappa = reports2(y_pred,y_test,name)
df_clas = classification
classification = str(classification)
confusion = str(confusion)
element_mean = np.mean(each_acc, axis=0)
element_std = np.std(each_acc, axis=0)
file_name = "/results/IDA_SVM_classification_"+name+".txt"

with open(file_name, 'w') as x_file:
    x_file.write('{} preprocessing time (s): '.format(preprocessing_time))
    x_file.write('\n')
    x_file.write('{} Training Time 2 (s)'.format(Train_time))
    x_file.write('\n')
    x_file.write('{} Test Time: (S)'.format(TEST_TIME))
    x_file.write('\n')
    x_file.write('{} Kappa accuracy (%)'.format(kappa))
    x_file.write('\n')
    x_file.write('{} Overall accuracy (%)'.format(oa))
    x_file.write('\n')
    x_file.write('{} Average accuracy (%)'.format(aa))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{}+{} mean_KAPPA ± std_KAPPA is:'.format(str(np.mean(kappa)), str(np.std(kappa))))
    x_file.write('\n')
    x_file.write('{}+{} mean_OA ± std_OA is: (%)'.format(str(np.mean(oa)), str(np.std(oa))))
    x_file.write('\n')
    x_file.write('{}+{} mean_AA ± std_AA is:'.format(str(np.mean(aa)), str(np.std(aa))))
    x_file.write('\n')
    x_file.write('{} AMean of all elements in confusion matrix: (%)'.format(str(element_mean)))
    x_file.write('\n')
    x_file.write('{} Standard deviation of all elements in confusion matrix: (%)'.format(str(element_std)))
    x_file.write('\n')
    x_file.write('==============================\n')
    x_file.write('{}classification Report:'.format(classification))
    x_file.write('\n')
    x_file.write('===============================\n')
    x_file.write('{} confusion Matrix:'.format(confusion))

2.384185791015625e-07


In [ ]:
l=[]
for i in range(q.shape[0]):
  if q.iloc[i, -1] == 0:
    l.append(0)
  else:
    l.append(svm.predict(q.iloc[i, :-1].values.reshape(1, -1)))

In [ ]:
import spectral
gt = np.array(q.iloc[:, -1]).reshape(h1, w1).astype('float')
ground_predict = spectral.imshow(classes = gt.astype(int), figsize =(7,7))

In [ ]:
import spectral
clmap = np.array(l).reshape(h1, w1).astype('float')
ground_predict = spectral.imshow(classes = clmap.astype(int), figsize =(7,7))

/tmp/ipykernel_38/4131742998.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  clmap = np.array(l).reshape(h1, w1).astype('float')


In [ ]:
import spectral
predict_image = spectral.imshow(classes = clmap.astype(int),figsize =(7,7))

In [ ]:
spectral.save_rgb("/results/IDA_SVM_"+name+".png", clmap.astype(int), colors=spectral.spy_colors) 